In [50]:
import torch
import torch.nn as nn

In [51]:
a = torch.Tensor([3])
b = torch.Tensor([4])

print(a.requires_grad)
print(b.requires_grad)

a.requires_grad_()
b.requires_grad_()

print(a.requires_grad)
print(b.requires_grad)

a = a.requires_grad_()
b = b.requires_grad_()

print(a.requires_grad)
print(b.requires_grad)

c = a * b
c.backward()

print(a.grad)
print(b.grad)

False
False
True
True
True
True
tensor([4.])
tensor([3.])


In [52]:
a = torch.Tensor([3])
b = torch.Tensor([4])
c = torch.Tensor([5])

a.requires_grad_()
b.requires_grad_()
c.requires_grad_()

d = a * b
e = c * d

e.backward()

print(a.grad)
print(b.grad)
print(c.grad)

tensor([20.])
tensor([15.])
tensor([12.])


In [53]:
a = torch.Tensor([3])
b = torch.Tensor([4])
c = torch.Tensor([5])

a.requires_grad_()
b.requires_grad_()
c.requires_grad_()

d = a * b
e = c * d

gradients = torch.autograd.grad(
    outputs=e,
    inputs=[a, b],
    create_graph=True,
    retain_graph=True,
)
print(gradients)
print(a.grad)
print(b.grad)
print(c.grad)

gradients = torch.autograd.grad(
    outputs=d,
    inputs=[a, b],
    grad_outputs=torch.ones(size=c.size()),
    create_graph=True,
    retain_graph=True,
)
print(gradients)
print(a.grad)
print(b.grad)
print(c.grad)

gradients = torch.autograd.grad(
    outputs=d,
    inputs=[a, b],
    grad_outputs=2*torch.ones(size=c.size()),
    create_graph=True,
    retain_graph=True,
)
print(gradients)
print(a.grad)
print(b.grad)
print(c.grad)

(tensor([20.], grad_fn=<MulBackward0>), tensor([15.], grad_fn=<MulBackward0>))
None
None
None
(tensor([4.], grad_fn=<MulBackward0>), tensor([3.], grad_fn=<MulBackward0>))
None
None
None
(tensor([8.], grad_fn=<MulBackward0>), tensor([6.], grad_fn=<MulBackward0>))
None
None
None


In [54]:
a = torch.Tensor([3])
b = torch.Tensor([4])
c = torch.Tensor([5])

a.requires_grad_()
b.requires_grad_()
c.requires_grad_()

d = a * b
e = c * d

torch.autograd.backward(tensors=e)

print(a.grad)
print(b.grad)
print(c.grad)

tensor([20.])
tensor([15.])
tensor([12.])


In [55]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


class Network(nn.Module):

    def __init__(self, input_size, hidden_size, output_size):
        super(Network, self).__init__()
        self.network = nn.Sequential(
             nn.Linear(input_size, hidden_size),
             nn.LeakyReLU(negative_slope=0.2, inplace=True),
             nn.Linear(hidden_size, output_size),
             nn.LeakyReLU(negative_slope=0.2, inplace=True)
        )

    def forward(self, x):
        return self.network(x)

batch_size = 256
input_size = 10   
hidden_size = 64   
output_size = 3 

x = torch.randn(batch_size, input_size)
target= torch.randn(batch_size, output_size)

model = Network(input_size, hidden_size, output_size)
print(model)

criterion = nn.MSELoss()

output = model(x)
loss = criterion(output, target)

loss.backward()

output = model(x)
loss = criterion(output, target)

loss.backward()

for parameter in model.parameters():
    print(parameter.grad.shape)

Network(
  (network): Sequential(
    (0): Linear(in_features=10, out_features=64, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Linear(in_features=64, out_features=3, bias=True)
    (3): LeakyReLU(negative_slope=0.2, inplace=True)
  )
)
torch.Size([64, 10])
torch.Size([64])
torch.Size([3, 64])
torch.Size([3])


In [56]:
image_size = 64
num_colors = 3
batch_size = 256
gen_hidden_dim = 128
label_dim = 40
latent_dim = 100
input_dim = latent_dim + label_dim

layer1 = nn.ConvTranspose2d(in_channels=input_dim, out_channels=gen_hidden_dim * 8, kernel_size=4, stride=1, padding=0)
layer2 = nn.ConvTranspose2d(in_channels=gen_hidden_dim * 8, out_channels=gen_hidden_dim * 4, kernel_size=4, stride=2, padding=1)
layer3 = nn.ConvTranspose2d(in_channels=gen_hidden_dim * 4, out_channels=gen_hidden_dim * 2, kernel_size=4, stride=2, padding=1)
layer4 = nn.ConvTranspose2d(in_channels=gen_hidden_dim * 2, out_channels=gen_hidden_dim, kernel_size=4, stride=2, padding=1)
layer5 = nn.ConvTranspose2d(in_channels=gen_hidden_dim, out_channels=num_colors, kernel_size=4, stride=2, padding=1)

x = torch.rand(size=(batch_size, input_dim))
x = x.view(x.size(0), x.size(1), 1, 1)
print(x.shape)

y1 = layer1(x)
print(y1.shape)

y2 = layer2(y1)
print(y2.shape)

y3 = layer3(y2)
print(y3.shape)

y4 = layer4(y3)
print(y4.shape)

y5 = layer5(y4)
print(y5.shape)

torch.Size([256, 140, 1, 1])
torch.Size([256, 1024, 4, 4])
torch.Size([256, 512, 8, 8])
torch.Size([256, 256, 16, 16])
torch.Size([256, 128, 32, 32])
torch.Size([256, 3, 64, 64])


In [57]:
image_size = 64
num_colors = 3
label_dim = 40
dis_hidden_dim = 128
batch_size = 256
input_dim = label_dim + num_colors

layer1 = nn.Sequential(
    nn.Conv2d(in_channels=input_dim, out_channels=dis_hidden_dim, kernel_size=4, stride=2, padding=1),
    nn.LeakyReLU(negative_slope=0.2, inplace=True),
)

layer2 = nn.Sequential(
    nn.Conv2d(in_channels=dis_hidden_dim, out_channels=dis_hidden_dim * 2, kernel_size=4, stride=2, padding=1),
    nn.LeakyReLU(negative_slope=0.2, inplace=True),
)

layer3 = nn.Sequential(
    nn.Conv2d(in_channels=dis_hidden_dim * 2, out_channels=dis_hidden_dim * 4, kernel_size=4, stride=2, padding=1),
    nn.LeakyReLU(negative_slope=0.2, inplace=True),
)

layer4 = nn.Sequential(
    nn.Conv2d(in_channels=dis_hidden_dim * 4, out_channels=dis_hidden_dim * 8, kernel_size=4, stride=2, padding=1),
    nn.LeakyReLU(negative_slope=0.2, inplace=True),
)

layer5 = nn.Sequential(
    nn.Conv2d(in_channels=dis_hidden_dim * 8, out_channels=1, kernel_size=4, stride=1, padding=0),
    nn.LeakyReLU(negative_slope=0.2, inplace=True)
)

x = torch.rand(size=(batch_size, input_dim, image_size, image_size))
print(x.shape)

y1 = layer1(x)
print(y1.shape)

y2 = layer2(y1)
print(y2.shape)

y3 = layer3(y2)
print(y3.shape)

y4 = layer4(y3)
print(y4.shape)

y5 = layer5(y4)
print(y5.shape)

torch.Size([256, 43, 64, 64])
torch.Size([256, 128, 32, 32])
torch.Size([256, 256, 16, 16])
torch.Size([256, 512, 8, 8])
torch.Size([256, 1024, 4, 4])
torch.Size([256, 1, 1, 1])
